### Libraries


In [7]:
import pandas as pd
import numpy as np
import requests
import time
import csv
from bs4 import BeautifulSoup

from IPython.display import display, HTML

### Functions

In [55]:
def get_ufcstats_links():
    url = 'http://ufcstats.com/statistics/events/completed'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    event_urls = []
    fight_details_urls = []

    for url in soup.select('.b-statistics__table-content a'):
        if url.has_attr('href'):
            event_urls.append(url['href'])
    
    for event_url in event_urls[:3]:
        response = requests.get(event_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        for i in soup.select('tr.b-fight-details__table-row'):
            if i.has_attr('data-link'):
                fight_details_urls.append(i['data-link'])

        time.sleep(2)

    return fight_details_urls
    

In [82]:
def extracting_details(match_url):
    response = requests.get(match_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    fighters = [name.text.strip() for name in soup.select(".b-fight-details__person-name")]
    result = [status.text.strip() for status in soup.select(".b-fight-details__person-status")]
    method = soup.select_one(".b-fight-details__text-item_first:-soup-contains('Method')")
    round = soup.select_one(".b-fight-details__text-item:-soup-contains('Round')")
    time = soup.select_one(".b-fight-details__text-item:-soup-contains('Time')")
    time_format = soup.select_one(".b-fight-details__text-item:-soup-contains('Time format')")
    referee = soup.select_one(".b-fight-details__text-item:-soup-contains('Referee')")
    details = [detail.text.strip() for detail in soup.select(".b-fight-details__text-item_first:-soup-contains('Details')")]

    fight_details_1 = {
        "Match_URL": match_url,

        "Fighter": fighters[0] if len(fighters) > 1 else "null",
        "Fighting_Against": fighters[1] if len(fighters) > 1 else "null",

        "Result": result[0] if len(result) > 1 else "null",
        "Method": method.text.strip().split("\n")[-1] if method else "null",
        "Round": round.text.strip().split("\n")[-1] if round else "null",
        "Time": time.text.strip().split("\n")[-1] if time else "null",
        "Time Format": time_format.text.strip().split("\n")[-1] if time_format else "null",
        "Referee": referee.text.strip().split("\n")[-1] if referee else "null",

        "Details": details if len(details) > 1 else "null"
    }      

    fight_details_2 = {
        "Match_URL": match_url,

        "Fighter": fighters[1] if len(fighters) > 1 else "null",
        "Fighting_Against": fighters[0] if len(fighters) > 1 else "null",

        "Result": result[1] if len(result) > 1 else "null",
        "Method": method.text.strip().split("\n")[-1] if method else "null",
        "Round": round.text.strip().split("\n")[-1] if round else "null",
        "Time": time.text.strip().split("\n")[-1] if time else "null",
        "Time Format": time_format.text.strip().split("\n")[-1] if time_format else "null",
        "Referee": referee.text.strip().split("\n")[-1] if referee else "null",

        "Details": details if len(details) > 1 else "null"
    }
    
    return fight_details_1, fight_details_2

### Main

In [81]:
def main():
    fight_details_list = get_ufcstats_links()
    list1 = []
    list2 = []
    fights = []

    for i in fight_details_list[26:29]:
        list1, list2 = extracting_details(i)
        fights.append(list1)
        fights.append(list2)
        time.sleep(2)

    df_fights = pd.DataFrame(fights)
    display(HTML(df_fights.to_html()))

    print('Done.')

In [80]:
if __name__ == '__main__':
    main()

,Match_URL,Fighter,Fighting_Against,Result,Method,Round,Time,Time Format,Referee,Details
0,http://ufcstats.com/fight-details/67f7163e536d62e2,Marvin Vettori,Roman Dolidze,L,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,null
1,http://ufcstats.com/fight-details/67f7163e536d62e2,Roman Dolidze,Marvin Vettori,W,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,null
2,http://ufcstats.com/fight-details/c83133ee114945ed,Chidi Njokuani,Elizeu Zaleski dos Santos,W,KO/TKO,2,2:19,3 Rnd (5-5-5),Mark Smith,null
3,http://ufcstats.com/fight-details/c83133ee114945ed,Elizeu Zaleski dos Santos,Chidi Njokuani,L,KO/TKO,2,2:19,3 Rnd (5-5-5),Mark Smith,null


Done.


In [ ]:
def get_event_links():
    url = 'http://ufcstats.com/statistics/events/completed'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    event_links = []

    for i in soup.select('.b-statistics__table-content a'): # Locating event links
        if i.has_attr('href'):
            event_links.append(i['href'])

    return event_links

def scrape_fight_data(event_url): # Scrapes individual fight details urls into a list
    response = requests.get(event_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    fight_details_url = []
    
    for i in soup.select('tr.b-fight-details__table-row'):
        if i.has_attr('data-link'):
            fight_details_url.append(i['data-link'])

    return fight_details_url